In [1]:
import pandas as pd 

In [4]:
df = pd.read_csv('combined_data.csv',parse_dates=['time'])

C:\Users\nikhi\AppData\Local\Temp\ipykernel_5352\4201758498.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('combined_data.csv',parse_dates=['time'])


In [5]:
df.head()

,time,xray_flux,flare_status
0,2001-03-01 00:00:00,3.478442e-07,NaN
1,2001-03-01 00:01:00,3.506659e-07,NaN
2,2001-03-01 00:02:00,3.533112e-07,NaN
3,2001-03-01 00:03:00,3.539179e-07,NaN
4,2001-03-01 00:04:00,3.518512e-07,NaN


In [6]:
def classify_flare(flux):
    if flux >= 1e-4:
        return 'X'
    elif flux >= 1e-5:
        return 'M'
    elif flux >= 1e-6:
        return 'C'
    else:
        return 'No Flare'

# Apply the function
df['flare_status'] = df['xray_flux'].apply(classify_flare)

# Check the distribution
print(df['flare_status'].value_counts())


flare_status
No Flare    9662681
C           1964258
M             78356
X              4785
Name: count, dtype: int64


In [7]:
df.head()

,time,xray_flux,flare_status
0,2001-03-01 00:00:00,3.478442e-07,No Flare
1,2001-03-01 00:01:00,3.506659e-07,No Flare
2,2001-03-01 00:02:00,3.533112e-07,No Flare
3,2001-03-01 00:03:00,3.539179e-07,No Flare
4,2001-03-01 00:04:00,3.518512e-07,No Flare


In [8]:
df['flare_status'].value_counts()


flare_status
No Flare    9662681
C           1964258
M             78356
X              4785
Name: count, dtype: int64

In [9]:
df['rolling_mean_5'] = df['xray_flux'].rolling(window=5).mean()
df['rolling_std_5'] = df['xray_flux'].rolling(window=5).std()
df['lag_1'] = df['xray_flux'].shift(1)


In [10]:
df.dropna(inplace=True)


In [11]:
print(df.isnull().sum())

time              0
xray_flux         0
flare_status      0
rolling_mean_5    0
rolling_std_5     0
lag_1             0
dtype: int64


In [12]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['label'] = le.fit_transform(df['flare_status'])


In [13]:
from sklearn.model_selection import train_test_split

features = ['xray_flux', 'rolling_mean_5', 'rolling_std_5', 'lag_1']
X = df[features]
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=50, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=50, random_state=42)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred_rf = rf_model.predict(X_test)

print(confusion_matrix(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf, target_names=le.classes_))


[[ 367041       0       0       0]
 [      0    9613       0       0]
 [      0       0 1608228       0]
 [      0       0       0     121]]
              precision    recall  f1-score   support

           C       1.00      1.00      1.00    367041
           M       1.00      1.00      1.00      9613
    No Flare       1.00      1.00      1.00   1608228
           X       1.00      1.00      1.00       121

    accuracy                           1.00   1985003
   macro avg       1.00      1.00      1.00   1985003
weighted avg       1.00      1.00      1.00   1985003



In [14]:

df['rolling_mean_10'] = df['xray_flux'].rolling(window=10).mean()
df['rolling_std_10'] = df['xray_flux'].rolling(window=10).std()

df['lag_1'] = df['xray_flux'].shift(1)
df['lag_3'] = df['xray_flux'].shift(3)
df['lag_5'] = df['xray_flux'].shift(5)

df['delta_flux'] = df['xray_flux'].diff()
df['delta_flux_3'] = df['xray_flux'].diff(3)

df.dropna(inplace=True)

features = ['rolling_mean_10', 'rolling_std_10', 'lag_1', 'lag_3', 'lag_5', 'delta_flux', 'delta_flux_3']
X = df[features]

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['label'] = le.fit_transform(df['flare_status'])
y = df['label']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_depth=10, n_jobs=-1, random_state=42)
rf.fit(X_train, y_train)


from sklearn.metrics import classification_report, confusion_matrix
y_pred = rf.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))


[[ 366782      72     187       0]
 [    131    9475       0       7]
 [    715       0 1607511       0]
 [      0       5       0     116]]
              precision    recall  f1-score   support

           C       1.00      1.00      1.00    367041
           M       0.99      0.99      0.99      9613
    No Flare       1.00      1.00      1.00   1608226
           X       0.94      0.96      0.95       121

    accuracy                           1.00   1985001
   macro avg       0.98      0.99      0.98   1985001
weighted avg       1.00      1.00      1.00   1985001



In [16]:
import joblib

joblib.dump(model, "solar_flare_rf_model.pkl")


['solar_flare_rf_model.pkl']

In [17]:
import pickle

with open("solar_flare_rf_model.pkl", "rb") as f:
    flare_model = pickle.load(f)
